In [1]:
!pip install transformers datasets torch scikit-learn numpy pandas tqdm



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
!pip install evaluate




[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
!pip install seqeval

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16185 sha256=f1d6023971578f2c5df37829455e1befa662e191764df6bb376d36164257cfe1
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\5f\b8\73\0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
# Hugging Face Transformers
from transformers import (AutoTokenizer, 
                          AutoModelForTokenClassification,
                          DataCollatorForTokenClassification,  
                          Trainer, 
                          TrainingArguments)

# PyTorch
import torch
from torch.utils.data import DataLoader, Dataset

from evaluate import load

metric = load("seqeval")  # Для оценки NER

# Для разделения данных и базовой обработки
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# Для отображения прогресса
from tqdm.notebook import tqdm


# Другие полезные импорты
import os
import random


In [26]:
def clean_file(input_file_path, output_file_path):
    with open(input_file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()
    
    print("Первые 10 строк файла до очистки:")
    print("\n".join(lines[:10]))  # Выводим первые 10 строк для отладки
    
    cleaned_lines = []
    
    for line in lines:
        line = line.strip()
        
        # Пропускаем строки с -DOCSTART- и пустые строки
        if line and not line.startswith("-DOCSTART-"):
            parts = line.split()
            
            # Проверка, что строка состоит хотя бы из двух частей: слово, возможно, дополнительные символы и метка
            if len(parts) >= 2:  # Возможно наличие лишних символов между словом и меткой
                word = parts[0]  # Первое слово
                tag = parts[-1]  # Последняя часть - метка
                cleaned_lines.append(f"{word} {tag}")
    
    print(f"Число строк после очистки: {len(cleaned_lines)}")  # Количество очищенных строк
    
    # Проверка, что очищенные строки не пустые
    if cleaned_lines:
        with open(output_file_path, "w", encoding="utf-8") as out_file:
            out_file.write("\n".join(cleaned_lines))
        print(f"Данные очищены и сохранены в {output_file_path}")
    else:
        print("Данные очищены, но файл пуст.")

# Пример использования
input_file_path = r"C:\Users\Lenovo\Downloads\project-2-at-2024-12-15-13-33-8d3d5fe8.conll"  # Исходный файл
output_file_path = r"C:\Users\Lenovo\Downloads\cleaned_data_v3.conll"  # Файл для очищенных данных

clean_file(input_file_path, output_file_path)


Первые 10 строк файла до очистки:
-DOCSTART- -X- O

In -X- _ B-O

the -X- _ I-O

winter -X- _ I-O

, -X- _ O

Kūh-e -X- _ B-B-Mountain

Senājen -X- _ B-I-Mountain

becomes -X- _ B-O

a -X- _ B-O

paradise -X- _ B-O

Число строк после очистки: 19835
Данные очищены и сохранены в C:\Users\Lenovo\Downloads\cleaned_data_v3.conll


In [29]:
def read_conll2003(file_path):
    sentences = []
    labels = []
    current_sentence = []
    current_labels = []

    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            line = line.strip()
            if line:  
                word, label = line.split()  # Разделяем слово и метку
                current_sentence.append(word)
                current_labels.append(label)
                
                if word == ".":
                    # Добавляем предложение и метки в список, если точка найдена
                    sentences.append(current_sentence)
                    labels.append(current_labels)
                    current_sentence = []
                    current_labels = []
                    
        if current_sentence:  # Добавляем последнее предложение, если файл не заканчивается пустой строкой
            sentences.append(current_sentence)
            labels.append(current_labels)

    return sentences, labels

# Чтение файла
file_path = r"C:\Users\Lenovo\Downloads\cleaned_data_v3.conll"
sentences, labels = read_conll2003(file_path)
print(f"Число предложений: {len(sentences)}")
#print(sentences)


Число предложений: 1097


In [30]:
train_texts, temp_texts, train_labels, temp_labels = train_test_split(sentences, labels, test_size=0.2, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(temp_texts, temp_labels, test_size=0.5, random_state=42)

print(f"Тренировочная выборка: {len(train_texts)}")
print(f"Валидационная выборка: {len(val_texts)}")
print(f"Тестовая выборка: {len(test_texts)}")

Тренировочная выборка: 877
Валидационная выборка: 110
Тестовая выборка: 110


In [31]:
# Загрузка токенизатора
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_and_align_labels(sentences, labels):
    tokenized_inputs = tokenizer(
        sentences,
        truncation=True,
        padding=True,
        is_split_into_words=True,
        return_tensors="pt"
    )

    aligned_labels = []
    for i, label in enumerate(labels):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_id = None
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)  # Игнорируем [CLS], [SEP], [PAD]
            elif word_id != previous_word_id:
                label_ids.append(label[word_id])
            else:
                label_ids.append(label[word_id] if label[word_id] == "I-Mountain" else -100)
            previous_word_id = word_id
        aligned_labels.append(label_ids)

    tokenized_inputs["labels"] = aligned_labels
    return tokenized_inputs

# Токенизация данных
train_encodings = tokenize_and_align_labels(train_texts, train_labels)
val_encodings = tokenize_and_align_labels(val_texts, val_labels)
test_encodings = tokenize_and_align_labels(test_texts, test_labels)

In [32]:

class NERDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

# Создание датасетов
train_dataset = NERDataset(train_encodings)
val_dataset = NERDataset(val_encodings)
test_dataset = NERDataset(test_encodings)


In [33]:
from torch.utils.data import DataLoader
from transformers import AdamW

In [37]:
print(labels[:10])

[['B-O', 'I-O', 'I-O', 'O', 'B-B-Mountain', 'B-I-Mountain', 'B-O', 'B-O', 'B-O', 'I-O', 'I-O', 'I-O', 'I-O', 'I-O', 'I-O', 'I-O', 'I-O', 'I-O', 'I-O', 'I-O', 'I-O'], ['B-O', 'B-O', 'O', 'B-O', 'B-O', 'B-O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-B-Mountain', 'B-I-Mountain', 'O'], ['B-B-Mountain', 'B-I-Mountain', 'B-I-Mountain', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-B-Mountain', 'B-I-Mountain', 'O', 'B-B-Mountain', 'B-I-Mountain', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'B-B-Mountain', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'B-B-Mountain', 'B-I-Mountain', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'B-B-Mountain', 'B-I-Mountain', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'B-B-Mountain', 'B-I-Mountain', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'B-B-Mountain', 'B-I-Mountain', '

In [38]:
# Преобразуем labels в одномерный список
labels_flat = [label for sublist in labels for label in sublist]

# Теперь создадим label2id
label2id = {label: i for i, label in enumerate(sorted(set(labels_flat)))}
id2label = {i: label for i, label in enumerate(sorted(set(labels_flat)))}


In [42]:
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(label2id))
optimizer = AdamW(model.parameters(), lr=5e-5)
# Подготовка DataLoader для тренировочных данных
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16)

# Функция для обучения модели
def train_model(model, train_dataloader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in tqdm(train_dataloader, desc="Training"):
        # Перенос данных на GPU (если доступен)
        batch = {k: v.to(device) for k, v in batch.items()}
        
        # Прогон через модель
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()

# Обратное распространение ошибки
        loss.backward()

        # Обновление параметров модели
        optimizer.step()
        optimizer.zero_grad()
    
    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Average training loss: {avg_train_loss}")

# Переход на GPU (если доступен)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# Обучение модели
train_model(model, train_dataloader, optimizer, device)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [1]:

!pip freeze > requirements.txt
